# **DATA ORGANIZATION** 🧺

this script will be used for organizing the data/feature engineering and writing other .csv/xslx files as needed

NOTE: the orginal datafile will not be saved in this repository as it contains confidential location information...each location will be assigned a number, and we will keep track of this list internally, however this number will not be used in the algorithms as a feature

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#imports and get raw data file
import pandas as pd
import numpy as np
import funcs
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
scaler = StandardScaler()


# note that 'private_name' is the associated secret number for the different locations
# df is importated from local machine to protect privacy
df = pd.read_csv('/Users/isaiah/Desktop/swl/nys pesticides/data/raw_data.csv')
df2 = pd.read_csv('/Users/isaiah/Desktop/swl/nys pesticides/data/raw_data.csv')
soil_params = pd.read_excel('/Users/isaiah/Desktop/swl/nys pesticides/data/Soil and Pestecide paramaters.xlsx', 
                            sheet_name='soil parameters')
pest_params = pd.read_excel('/Users/isaiah/Desktop/swl/nys pesticides/data/Soil and Pestecide paramaters.xlsx', 
                            sheet_name='Pestecide parameters')
t1 = pd.read_excel('../data/table1.xlsx')

In [11]:
'''
FILTERING PROCESS SUMMARY:

1. { Fill NaN results as 0 } - A NaN result indicates a nondetect in the case of pesticides.

2. { Fill NaN detection limits } - Many detection limits were not included in the original datafile if there was no detection. However, all
detection limits for scenario were included at least once somewhere else in the dataset. The values are found by searching for the same
pesticide and sample year.

3. { Find the actual pesticide tests } - Many of the tests are for other soil/water parameters (pH, electrical conductivity, etc) 
so we want to extract just pesticide tests. The dataset is filtered by a logical OR statement of 3 separate AND statements:
  - Column 'wasused' does not equal 'no'  AND  column 'wasused' is not NaN || identifies pesticides that were used or maybe used at the site
  or a neighboring site

OR

  - Column 'koc' is not NaN AND column 'result' is > 0 || identifies detections despite 'wasused' being 'no' or NaN

OR

- Column 'kfoc' is not NaN AND column 'result' is > 0 || same idea as previous statement, but in case the 'koc' value is not listed

4. { Filter out NaN drainage classes and soil halflives}  - Some drainage classes and soil halflives were not included in the dataset.

5. { Filter out Sulfur } - Sulfur had some odd results and was removed until furthur notice.

6. { Filter out certain location types } - We are uninterested in loctype 'Pond', 'Categorical - potable', and 'Long term' as these were ancillary tests or not enough information 
  is known about the testing area

'''

# 1. fill na results to 0
df['result'] = df['result'].fillna(0)
pd.to_numeric(df['result'])

# 2. fill all detection limits
df['detlimit'] = df['detlimit'].astype(str)
for idx, row in df.iterrows():
    if row['detlimit'] == 'nan':
        parameter = row['parameter']
        detlimit = df[(df['detlimit'] != 'nan') & (df['parameter'] == parameter)]

        # fill limit if found elsewhere
        if len(detlimit) > 0:
            year = row['sampdate'][0:4]
            for idx2, row2 in detlimit.iterrows():
                if row2['sampdate'][0:4] == year:
                    df.loc[idx, 'detlimit'] = detlimit.loc[idx2,'detlimit']
                    break

df['detlimit'] = df['detlimit'].apply(lambda x: x.replace('*',''))
df['detlimit'] = df['detlimit'].apply(lambda x: x.replace('>',''))
df['detlimit'] = df['detlimit'].apply(lambda x: x.replace('<',''))
df['detlimit'] = df['detlimit'].apply(lambda x: x.replace('?',''))
df['detlimit'] = df['detlimit'].astype(float)

# 3/4/5. find actual tests, filter drainage class and soil halflives, and remove sulfur
df_tests = df[np.logical_or((df['wasused'] != 'no') &  (df['wasused'].notnull()), df['koc'].notnull() & df['result'] > 0, df['kfoc'].notnull() & df['result'] > 0)]
df_tests = df_tests[df_tests['drainage_class'].notnull() & df_tests['soil_halflife'].notnull()]
df_tests = df_tests[df_tests['parameter'] != 'Sulfur']

# 6. remove certain loctypes
df_tests['loctype'] = df_tests['loctype'].apply(lambda x: x.strip())
df_tests = df_tests[df_tests['loctype'] != 'Pond']
df_tests = df_tests[df_tests['loctype'] != 'Categorical - potable']
#df_tests = df_tests[df_tests['loctype'] != 'Long term']
df_tests['site_code'] = df_tests['site_code'].str.lower()
df_tests['loccode'] = df_tests['loccode'].str.lower()
soil_params['Identify'] = soil_params['Identify'].str.lower()
soil_params['loccode'] = soil_params['loccode'].str.lower()


In [12]:
'''
FILTERING PROCESS CONTINUED

7. { Find usable tests for TGUS equation } - Our current documentation is on pesticide/site/location parameters for the TGUS equation 
is not completed, so the nonusable tests are filtered out. Documentation is found in the df's 'pest_params' and 'soil_params'.

'''
# 7. filter by usable TGUS tests
#documents which parameters/site codes/location codes from usable results are not in pest_params, for personal use
params_not_in = [] 
scode_not_in = []
lcode_not_in = []

kocs = [] #partitioning coefficients
apps = [] #pesticide applied amount
oms = [] #percent soil organic matter
bulks = [] #soil bulk densities
sand = [] #soil percent sand
silt = [] #soil percent silt
clay = [] #soil percent clay

# first get rid of non-included pesticides and sites
for idx, row in df_tests.iterrows():
    param = row['parameter']
    scode = row['site_code']
    lcode = row['loccode']
    if (param not in np.array(pest_params.loc[:,'parameter']) and param not in params_not_in):
        params_not_in.append(param)
        df_tests = df_tests[df_tests['parameter'] != param]
    elif (scode not in np.array(soil_params.loc[:,'Identify']) and scode not in scode_not_in):
        scode_not_in.append(scode)
        df_tests = df_tests[df_tests['site_code'] != scode]
    elif (lcode not in np.array(soil_params.loc[:,'loccode']) and lcode not in lcode_not_in):
        lcode_not_in.append(lcode)
        df_tests = df_tests[(df_tests['site_code'] != scode) & (df_tests['loccode'] != lcode)]

# loop thru again to get necessary data
for idx, row in df_tests.iterrows():
    param = row['parameter']
    scode = row['site_code']
    lcode = row['loccode']
    kocs += [float(pest_params[pest_params['parameter'] == param].loc[:,'KOC (m^3/Mg)=(cm^3/gr)'])]
    apps += [float(pest_params[pest_params['parameter'] == param].loc[:,'apllayd amoint (ug/Hectar)'])]
    oms += [float(soil_params[(soil_params['Identify'] == scode) & (soil_params['loccode'] == lcode)].loc[:,'Organic matter (%)'])]
    bulks += [float(soil_params[(soil_params['Identify'] == scode) & (soil_params['loccode'] == lcode)].loc[:,'Bulk density (gr/cm3)'])]
    sand += [float(soil_params[(soil_params['Identify'] == scode) & (soil_params['loccode'] == lcode)].loc[:,'Sand %'])]
    silt += [float(soil_params[(soil_params['Identify'] == scode) & (soil_params['loccode'] == lcode)].loc[:,'Silt %'])]
    clay += [float(soil_params[(soil_params['Identify'] == scode) & (soil_params['loccode'] == lcode)].loc[:,'Clay %'])]


# append necessary data
df_tests.loc[:,['koc [m^3/Mg]','application rates [mg/m^2]', 'organic matter [%]', 'bulk density [Mg/m^3]',
                 'Sand %', 'Silt %', 'Clay %']] = np.array([kocs, apps, oms, bulks, sand, silt, clay]).T

#adjust inorganic matter %
for idx, row in df_tests.iterrows():
    df_tests.loc[idx,'True Sand'] = round(row['Sand %'] / (100 + row['organic matter [%]']) * 100, 2)
    df_tests.loc[idx,'True Silt'] = round(row['Silt %'] / (100 + row['organic matter [%]']) * 100, 2)
    df_tests.loc[idx,'True Clay'] = round(row['Clay %'] / (100 + row['organic matter [%]']) * 100, 2)

# convert necessary units for TGUS equation and isolate high bulk densities
df_tests['application rates [mg/m^2]'] = df_tests['application rates [mg/m^2]'] * 0.0000001


#df_tests = df_tests[df_tests['bulk density [Mg/m^3]'] >= 1] USE IF DOING MINERAL SIZES




In [13]:
'''
NO LONGER USING !!!!!!!!!

IMPORTANT CONCEPT

- theoretically, the organic carbon-water partition coefficient ('koc' column) and the organic carbon-water normalized Freundlich distribution 
  coefficient will be treated as the same

- this loop combines the columns, choosing koc first if it is available


pcoef = []
for idx, row in df_tests.iterrows():
    if row['koc'] > 0 :
        pcoef += [float(row['koc'])]
    else :
        pcoef += [float(row['kfoc'])]

df_tests['pcoef'] = pcoef
'''

"\nNO LONGER USING !!!!!!!!!\n\nIMPORTANT CONCEPT\n\n- theoretically, the organic carbon-water partition coefficient ('koc' column) and the organic carbon-water normalized Freundlich distribution \n  coefficient will be treated as the same\n\n- this loop combines the columns, choosing koc first if it is available\n\n\npcoef = []\nfor idx, row in df_tests.iterrows():\n    if row['koc'] > 0 :\n        pcoef += [float(row['koc'])]\n    else :\n        pcoef += [float(row['kfoc'])]\n\ndf_tests['pcoef'] = pcoef\n"

In [14]:
'''
- extract all current columns of potential interest to be put into algorithms...NOT FINAl!!!!!!!!!!!!!!

- other minor fixes
'''
col_list = ['private_name', 'loctype', 'aquifer_vulnerability', 'drainage_class', 'detlimit', 'sampdate', 'parameter','gus', 
            'soil_halflife', 'simphalflife', 'morehalflives', 'koc [m^3/Mg]','application rates [mg/m^2]', 'organic matter [%]', 
            'bulk density [Mg/m^3]', 'Sand %', 'True Sand', 'Silt %', 'True Silt', 'Clay %', 'True Clay', 'simpsorp', 'simpsorp2', 'result', 'simpresult']

#get all columns of interest
df_cols = df_tests.loc[:, col_list]

#replace all instances of 'well drained' to 'Well drained'
df_cols.replace(to_replace='well drained', value='Well drained', inplace = True)



In [15]:
# reset index
df_adjusted = df_cols.reset_index().iloc[:,1:]

# add a 'detected' column if result > 0
# 1 if detected, -1 if not
for idx, row in df_adjusted.iterrows():
    if df_adjusted.loc[idx, 'result'] > 0:
        df_adjusted.loc[idx, 'detected'] = 1
    else:
        df_adjusted.loc[idx, 'detected'] = -1

In [16]:
'''
THIS IS NOT BEING DONE UNTIL FURTHER NOTICE!

IMPORTANT CONCEPT:

- at many testing sites, samples were taken in both the downgradient and upgradient groundwater of the pesticide-treated area...
  these are distinguished by 'Categorical - upgradient' and'Categorical - downgradient'...'Categorical - up and downgradient' indicates
  one site where the test was both upgradient of one treated area and downgradient of another

- tests were done at upgradient sites to find out if pesticides were in the already in the groundwater NOT as a result of the land-owners'
  application...this could be the result of a neighboring property apply pesticides, for example...if the same pesticide is detected downgradient 
  and upgradient of the pesticide application area, then the upgradient value should be subtracted from the downgradient value to get a better
  representation of what is happening with land-owners' pesticides

- this loop identifies upgradient/downgradient tests on the same sampling date and subtracts the upgradient result from the downgradient



# reset index
df_reset = df_cols.reset_index().iloc[:,1:]




for idx, row in df_reset.iterrows():
    # find 'upgradient' or 'up and downgradient' test on same date for same parameter in same location
    if row['loctype'] in ['Categorical - downgradient','Categorical - up and downgradient']:
        sampdate = row['sampdate']
        parameter = row['parameter']
        loctype = row['loctype']
        name = row['private_name']
        upgradient = df_reset[(df_reset['private_name'] == name) & (df_reset['sampdate'] == sampdate) & (df_reset['loctype'] > loctype) & (df_reset['parameter'] == parameter)]

    # if test has both 'upgradient' and 'up and downgradient' samples, then subtract just the 'up and downgradient'
    # when upgradient is created, it puts 'up and downgradient' results first, so we can just subtract out first index of whatever upgradient is
    if len(upgradient) > 0:
      df_reset.loc[idx, 'result'] -= upgradient.loc[upgradient.index[0],'result']

# now extract out just the downgradient tests of interests
# UNDECIDED ON THIS! for now just copy df_reset
#df_adjusted = df_reset[(df_reset['loctype'] != 'Categorical - upgradient') & (df_reset['loctype'] != 'Categorical - up and downgradient') ]

df_adjusted = df_reset.loc[:,:]


# add a 'detected' column if result > 0
# 1 if detected, -1 if not
for idx, row in df_adjusted.iterrows():
    if df_adjusted.loc[idx, 'result'] > 0:
        df_adjusted.loc[idx, 'detected'] = 1
    else:
        df_adjusted.loc[idx, 'detected'] = -1
'''


"\nTHIS IS NOT BEING DONE UNTIL FURTHER NOTICE!\n\nIMPORTANT CONCEPT:\n\n- at many testing sites, samples were taken in both the downgradient and upgradient groundwater of the pesticide-treated area...\n  these are distinguished by 'Categorical - upgradient' and'Categorical - downgradient'...'Categorical - up and downgradient' indicates\n  one site where the test was both upgradient of one treated area and downgradient of another\n\n- tests were done at upgradient sites to find out if pesticides were in the already in the groundwater NOT as a result of the land-owners'\n  application...this could be the result of a neighboring property apply pesticides, for example...if the same pesticide is detected downgradient \n  and upgradient of the pesticide application area, then the upgradient value should be subtracted from the downgradient value to get a better\n  representation of what is happening with land-owners' pesticides\n\n- this loop identifies upgradient/downgradient tests on the s

In [17]:
# ONLY NEEDED IF LOCTYPE SUBTRACTION IS DONE
# reset index again
#df_adjusted = df_adjusted.reset_index().iloc[:,1:]

In [18]:
'''
TGUS EQUATION

- SWL lab members are currently deriving a new theoretical groundwater ubiquity score (TGUS) to be compared to typically used groundwater ubiquity
  score (GUS) derived by Gustafson et al., 1989

- equation is written in 'funcs.py' file and called for each test

'''
for idx, row in df_adjusted.iterrows():
  tgus = funcs.tgus(row['soil_halflife'], row['application rates [mg/m^2]'], row['detlimit'], row['organic matter [%]'],
                    row['bulk density [Mg/m^3]'], row['koc [m^3/Mg]'])
  df_adjusted.loc[idx, 'tgus'] = tgus




In [19]:
''' 
FEATURE ENGINEERING:

- creates final dataframe of all ordinal/one-hot data and raw data

- recall: if doing SVM or something that isn't trees, need to do scale raw data
'''
# setup final dataframe
# for now, working with all raw numbers and not pre-decided categories
cat_cols = ['aquifer_vulnerability','drainage_class']
raw_cols = ['gus','tgus','soil_halflife', 'koc [m^3/Mg]','detlimit', 
            'Sand %', 'Silt %', 'Clay %', 'True Sand', 'True Silt', 'True Clay']

# normalize raw values ONLY IF NOT DOING TREES
#norm = scaler.fit_transform(df_adjusted.loc[:, raw_cols])
#norm = round(pd.DataFrame(norm, columns = raw_cols), 3)

# ordinal encoder for categoricals
df_cats= funcs.ordinal(df_adjusted, 'drainage_class')
df_cats = funcs.ordinal(df_cats, 'aquifer_vulnerability')
cats = df_cats.loc[:, cat_cols]
'''
# onehot categorical
df_onehot = funcs.onehot(df=df_adjusted, columns = onehot_cols)
df_final = pd.concat([df_onehot, norm], axis = 1)
'''

# combine, then append offset and re-add detected column
df_final = pd.concat([cats, df_adjusted.loc[:, raw_cols]], axis = 1)
df_final['offset'] = np.ones((df_adjusted.shape[0]))
df_final['detected'] = df_adjusted['detected']



In [20]:
'''
SEPARATE DATAFRAMES

- 'df_final' is used to create different datasets to compare

'''
df_a = df_final.loc[:, ~df_final.columns.isin(['True Sand','True Silt', 'True Clay','Sand %', 'Silt %', 'Clay %'])]#'soil_halflife', 'koc [m^3/Mg]','detlimit'])]
df_b = df_final.loc[:, ~df_final.columns.isin(['True Sand','True Silt', 'True Clay','tgus','soil_halflife', 'koc [m^3/Mg]', 'Sand %', 'Silt %', 'Clay %' ])]# 'aquifer_vulnerability', 'drainage_class','detlimit'
df_c = df_final.loc[:, ~df_final.columns.isin(['True Sand','True Silt', 'True Clay','gus','soil_halflife', 'koc [m^3/Mg]',  'Sand %', 'Clay %', 'Silt %'])]#, 'aquifer_vulnerability', 'drainage_class','detlimit'
#df_raw = df_final.loc[:, ~df_final.columns.isin(['tgus', 'tgus*', 'gus','detlimit'])]
df_d = df_final.loc[:, ~df_final.columns.isin(['gus','tgus','True Sand','True Silt', 'True Clay', 'Sand %', 'Silt %', 'Clay %'])]#


In [21]:
#write df_final as csv for future use
df_a.to_csv(path_or_buf = '../data/df_a.csv', sep = ',')
df_b.to_csv(path_or_buf = '../data/df_b.csv', sep = ',')
df_c.to_csv(path_or_buf = '../data/df_c.csv', sep = ',')
#df_raw.to_csv(path_or_buf = '../data/df_raw.csv', sep = ',')
df_d.to_csv(path_or_buf = '../data/df_d.csv', sep = ',')